### Process Area Assessment
This notebook is the third notebook in the resourceAssessment workflow. In this notebook we save export our area assessment features into a postgreSQL database, 

In [1]:
import os
import sys
import csv
import pandas as pd
import numpy as np
import geopandas as gpd
from sqlalchemy import create_engine
from sqlalchemy import inspect

In [2]:
# write connection string
host = "localhost"
database = "asset"
user = os.getenv('SQL_USER')
password = os.getenv('SQL_PASSWORD')
connection_string = f"postgresql://{user}:{password}@{host}/{database}"

In [3]:
# create sqlalchemy engine
engine = create_engine(connection_string)

In [4]:
# set folder path and output path for .csv files
inputFolder = "C:\\Users\\Zachary\\ASSET\\resourceAssessment\\analysis\\data"
folder_path = os.path.join(inputFolder, "shapefiles")
outputFolder = os.path.join(inputFolder, "csv")

# create ouput folder if it doesn't exist
if not os.path.exists(outputFolder):
    os.makedirs(outputFolder)

In [5]:
# list to store table names
table_names = []

# Iterate through the files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".shp"):
        # Create the full path to the shapefile (without extension)
        file_path = os.path.join(folder_path, os.path.splitext(filename)[0] + ".shp")

        # Read the shapefile using geopandas
        gdf = gpd.read_file(file_path)

        # Convert column names to lowercase
        gdf.columns = [column.lower() for column in gdf.columns]

        # Export the GeoDataFrame into PostgreSQL
        table_name = os.path.splitext(filename)[0].lower()  # Use the filename as the table name

        # add table name to list
        table_names.append(table_name)

        # add table to PostgreSQL
        gdf.to_postgis(name=table_name, con=engine, if_exists="replace")

In [6]:
print(table_names)

['sl1_solar_cesm2_areaassessment_full_reproject', 'sl1_solar_cesm2_areaassessment_wecc_reproject', 'sl1_solar_era5_areaassessment_full_reproject', 'sl1_solar_era5_areaassessment_wecc_reproject', 'sl1_wind_cesm2_areaassessment_full_reproject', 'sl1_wind_cesm2_areaassessment_wecc_reproject', 'sl1_wind_era5_areaassessment_full_reproject', 'sl1_wind_era5_areaassessment_wecc_reproject']


In [7]:
# Iterate over the table names, update tables and export to csv

for table_name in table_names:
    query = '''ALTER TABLE {}
                ADD COLUMN geom_text varchar;

                UPDATE {}
                SET geom_text = ST_AsText(ST_Force2d(geometry), 4);

                ALTER TABLE {}
                ADD COLUMN geom_centroid geometry;

                UPDATE {}
                SET geom_centroid = ST_Centroid(ST_Force2d(geometry));

                ALTER TABLE {}
                ADD COLUMN centroid_text varchar;

                UPDATE {}
                SET centroid_text = ST_AsText(geom_centroid, 4);

                SELECT * FROM {}
                ORDER BY objectid'''.format(table_name, table_name, table_name, table_name, table_name, table_name, table_name)

    result = engine.execute(query)
    rows = result.fetchall()
    
    # Define the CSV file path within the "csv" folder
    csv_file_path = os.path.join(outputFolder, f"{table_name}.csv")

    # Export the query result to a CSV file
    with open(csv_file_path, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)

       # Write the header
        header = result.keys()
        csv_writer.writerow(header)
        csv_writer.writerows(rows)  # Write the data rows

    
    print(f"Table {table_name} exported to CSV: {csv_file_path}")

    print()  # Add a newline between tables

print("Query execution completed!")

Table sl1_solar_cesm2_areaassessment_full_reproject exported to CSV: C:\Users\Zachary\ASSET\resourceAssessment\analysis\data\csv\sl1_solar_cesm2_areaassessment_full_reproject.csv

Table sl1_solar_cesm2_areaassessment_wecc_reproject exported to CSV: C:\Users\Zachary\ASSET\resourceAssessment\analysis\data\csv\sl1_solar_cesm2_areaassessment_wecc_reproject.csv

Table sl1_solar_era5_areaassessment_full_reproject exported to CSV: C:\Users\Zachary\ASSET\resourceAssessment\analysis\data\csv\sl1_solar_era5_areaassessment_full_reproject.csv

Table sl1_solar_era5_areaassessment_wecc_reproject exported to CSV: C:\Users\Zachary\ASSET\resourceAssessment\analysis\data\csv\sl1_solar_era5_areaassessment_wecc_reproject.csv

Table sl1_wind_cesm2_areaassessment_full_reproject exported to CSV: C:\Users\Zachary\ASSET\resourceAssessment\analysis\data\csv\sl1_wind_cesm2_areaassessment_full_reproject.csv

Table sl1_wind_cesm2_areaassessment_wecc_reproject exported to CSV: C:\Users\Zachary\ASSET\resourceAssessm